In [2]:
from bs4 import BeautifulSoup

# Liste des chemins vers les fichiers HTML
file_paths = ['html/html_ligne8.txt', 'html/html_tf1.txt', 'html/html_emoji.txt']

# Initialiser une liste pour cumuler tous les divs
divs = []

# Parcourir chaque fichier dans la liste
for file_path in file_paths:
    # Lire le contenu du fichier HTML
    with open(file_path, 'r', encoding='utf-8') as file:
        html_content = file.read()

    # Parser le contenu HTML avec BeautifulSoup
    soup = BeautifulSoup(html_content, 'html.parser')

    # Extraire toutes les divs ayant data-testid="tweetText" et les ajouter à la liste cumulée
    divs.extend(soup.find_all('div', {'data-testid': 'tweetText'}))

L_tweet = []

# Extraire et imprimer le texte de chaque div
for div in divs:
    tweet_parts = []

    for element in div.children:
        if element.name == 'img':
            tweet_parts.append(element['alt'])  # Ajouter l'emoji
        else:
            tweet_parts.append(element.get_text())  # Ajouter le texte

    tweet_text = ''.join(tweet_parts)
    L_tweet.append(tweet_text)

    print(tweet_text)
    print("-" * 80)  # Séparateur pour chaque tweet


[#31décembre] Pour votre soirée du Nouvel An, certaines lignes circuleront toute la nuit ! ✨ Pour connaître le détail des horaires et des stations ouvertes, rendez-vous par ici ➡️ https://ratp.fr/nouvel-an
#RATP pour @idfmobilites
--------------------------------------------------------------------------------
❌ Le trafic est interrompu entre La Motte-Picquet - Grenelle et Balard et reprendra vers 11:00. Cela fait suite à un bagage oublié dans un train à Balard. #ligne8 #RATP
--------------------------------------------------------------------------------
⚠️ Le trafic reprend mais reste perturbé sur l'ensemble de la ligne, en répercussion d'un bagage oublié dans un train à Balard. #ligne8 #RATP
--------------------------------------------------------------------------------
✅ Retour à un trafic régulier sur l'ensemble de la ligne (bagage oublié dans un train). #ligne8 #RATP
--------------------------------------------------------------------------------
❌le trafic est interrompu entre 

In [3]:
len(L_tweet)

819

In [5]:
import re
from langdetect import detect, LangDetectException

# Expression régulière pour détecter les emojis
emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F700-\U0001F77F"  # alchemical symbols
                           u"\U0001F780-\U0001F7FF"  # Geometric Shapes Extended
                           u"\U0001F800-\U0001F8FF"  # Supplemental Arrows-C
                           u"\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
                           u"\U0001FA00-\U0001FA6F"  # Chess Symbols
                           u"\U0001FA70-\U0001FAFF"  # Symbols and Pictographs Extended-A
                           u"\U0001FB00-\U0001FBFF"  # Symbols for Legacy Computing
                           u"\U00002500-\U00002BEF"  # various symbols
                           u"\U0001F1E6-\U0001F1FF"  # flags (iOS)
                           u"\U0001F191-\U0001F251"  # enclosed characters
                           u"\U0001F004"  # Mahjong tile red dragon
                           u"\U0001F0CF"  # Playing card black joker
                           u"\U00002600-\U000026FF"  # miscellaneous symbols
                           u"\U00002700-\U000027BF"  # Dingbats
                           u"\U0001F18E"  # NEGATIVE SQUARED AB
                           u"\U0001F191-\U0001F251"  # Enclosed Characters
                           u"\U0001F201-\U0001F2FF"  # Enclosed Ideographic Supplement
                           u"\u02B50"  # White Medium Star
                           u"\U0001F004"  # Mahjong Tile Red Dragon
                           "]+", flags=re.UNICODE)

L_sentences = []

for post_content in L_tweet:
    words = post_content.split()
    for i, word in enumerate(words):
        if emoji_pattern.search(word):
            if i == 0:  # Emoji at the start of the line
                phrase = ' '.join(words[i:])
                end = re.search(r'[,.!?]', phrase)
                if end:
                    phrase = phrase[:end.end()]
            else:  # Emoji not at the start
                start = max(0, i - 5)
                phrase = ' '.join(words[start:i+1])
                if re.search(r'[,.!?]', phrase):
                    phrase = ' '.join(words[start:i])
            # Language detection
            try:
                if detect(phrase) == 'fr':
                    print(phrase)
                    L_sentences.append(phrase)
                    print("-" * 80)
            except LangDetectException:
                continue

print(L_sentences)


circuleront toute la nuit !
--------------------------------------------------------------------------------
stations ouvertes, rendez-vous par ici
--------------------------------------------------------------------------------
❌ Le trafic est interrompu entre La Motte-Picquet - Grenelle et Balard et reprendra vers 11:00.
--------------------------------------------------------------------------------
⚠️ Le trafic reprend mais reste perturbé sur l'ensemble de la ligne,
--------------------------------------------------------------------------------
✅ Retour à un trafic régulier sur l'ensemble de la ligne (bagage oublié dans un train).
--------------------------------------------------------------------------------
❌le trafic est interrompu entre Reuilly-Diderot et Pointe du Lac et reprendra vers 20:30.
--------------------------------------------------------------------------------
du Lac et reprendra vers
-------------------------------------------------------------------------------

In [6]:
len(L_sentences)

951

In [7]:
import re

# Fonction pour extraire un seul emoji selon les règles spécifiées
def extract_single_emoji(sentence):
    # Détecter tous les emojis dans la phrase
    emojis = emoji_pattern.findall(sentence)

    if not emojis:
        return None  # Retourner None s'il n'y a pas d'emoji dans la phrase

    emoji_selected = emojis[0]

    words = sentence.split()

    for i in range(len(words)-1, -1, -1):
        word = words[i]
        if (word in emojis):
            if words[i-1] in emojis:
                continue
            else:
                emoji_selected = word
                break
    
    sentence_without_emojis = emoji_pattern.sub(r'', sentence)
    new_sentence = re.sub(r'\s+', ' ', sentence_without_emojis).strip() + ' ' + emoji_selected    
    return new_sentence

# Filtrer les phrases de L_sentences pour ne garder qu'un seul emoji par ligne
filtered_sentences = []
for sentence in L_sentences:
    new_sentence = extract_single_emoji(sentence)
    if new_sentence:  # Ajouter seulement si la phrase contient un emoji
        filtered_sentences.append(new_sentence)
        print(new_sentence)
        print("-" * 80)

L_sentences = filtered_sentences

Le trafic est interrompu entre La Motte-Picquet - Grenelle et Balard et reprendra vers 11:. ❌
--------------------------------------------------------------------------------
️ Le trafic reprend mais reste perturbé sur l'ensemble de la ligne, ⚠
--------------------------------------------------------------------------------
Retour à un trafic régulier sur l'ensemble de la ligne (bagage oublié dans un train). ✅
--------------------------------------------------------------------------------
le trafic est interrompu entre Reuilly-Diderot et Pointe du Lac et reprendra vers 2:3. ❌
--------------------------------------------------------------------------------
️Le trafic reprend mais reste perturbé sur l'ensemble de la ligne, ⚠
--------------------------------------------------------------------------------
️Le trafic reprend mais reste perturbé sur l'ensemble de la ligne, ⚠
--------------------------------------------------------------------------------
️ Le trafic est perturbé en directi

In [8]:
len(L_sentences)

704

In [9]:
# Eliminer les doublons dans L_sentences
L_sentences_unique = list(set(L_sentences))

# Afficher les phrases uniques
for sentence in L_sentences_unique:
    print(sentence)
    print("-" * 80)

L_sentences = L_sentences_unique

vous souhaitent une belle année 225 0
--------------------------------------------------------------------------------
@Clemence_Guette qui prend un cours de droit par une journaliste spécialisée sur la Constitution en plein direct #Levenement #Macron2h 🔴
--------------------------------------------------------------------------------
INFO BFMTV Cyclone à Mayotte: Marine Le Pen va se rendre sur place dimanche et lundi https://l. 🔴
--------------------------------------------------------------------------------
l’a fait encore My presi 😌
--------------------------------------------------------------------------------
si vous insultez trop le 9 😒
--------------------------------------------------------------------------------
avc mon diesel pourri critair 4 0
--------------------------------------------------------------------------------
une rafale de vent surprenante 😥
--------------------------------------------------------------------------------
sa (album,zénith et plein 🤯
---------

In [10]:
len(L_sentences)

679

In [11]:
import csv
import re

# Exemple de liste de phrases
# phrases = [
#     "✅J'avale une barre de céréale protéiné🍫 et je commence mon footing 🏃",
#     "C'est l'heure du café ☕ et du croissant 🥐 pour bien commencer la journée! 🌞",
#     "Il pleut 🌧️ mais je reste positif 😃 et je continue ma journée!",
#     "Enormément de temps perdu⏱⏱;➡️️️"
# ]

phrases = L_sentences

# Expression régulière pour détecter les emojis
emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F700-\U0001F77F"  # alchemical symbols
                           u"\U0001F780-\U0001F7FF"  # Geometric Shapes Extended
                           u"\U0001F800-\U0001F8FF"  # Supplemental Arrows-C
                           u"\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
                           u"\U0001FA00-\U0001FA6F"  # Chess Symbols
                           u"\U0001FA70-\U0001FAFF"  # Symbols and Pictographs Extended-A
                           u"\U0001FB00-\U0001FBFF"  # Symbols for Legacy Computing
                           u"\U00002500-\U00002BEF"  # various symbols
                           u"\U0001F1E6-\U0001F1FF"  # flags (iOS)
                           u"\U0001F191-\U0001F251"  # enclosed characters
                           u"\U0001F004"  # Mahjong tile red dragon
                           u"\U0001F0CF"  # Playing card black joker
                           u"\U00002600-\U000026FF"  # miscellaneous symbols
                           u"\U00002700-\U000027BF"  # Dingbats
                           u"\U0001F18E"  # NEGATIVE SQUARED AB
                           u"\U0001F191-\U0001F251"  # Enclosed Characters
                           u"\U0001F201-\U0001F2FF"  # Enclosed Ideographic Supplement
                           u"\U00002B50"  # White Medium Star
                           u"\U0001F004"  # Mahjong Tile Red Dragon
                           "]+", flags=re.UNICODE)

# Ouvrir le fichier CSV en mode append pour ajouter des lignes sans écraser le contenu existant
with open('phrases_emojis.csv', 'a', newline='', encoding='utf-8') as csvfile:
    csvwriter = csv.writer(csvfile, delimiter=',', quoting=csv.QUOTE_ALL)

    # Vérifiez si le fichier est vide pour ajouter l'en-tête
    csvfile.seek(0, 2)
    if csvfile.tell() == 0:
        csvwriter.writerow(['Phrase', 'Emojis'])

    # Parcourir chaque phrase dans la liste
    for phrase in phrases:
        # Extraire les emojis de la phrase
        emojis = ''.join(emoji_pattern.findall(phrase))

        # Vérifier si des emojis ont été trouvés
        if emojis:
            # Retirer les emojis de la phrase pour obtenir le texte pur
            clean_phrase = emoji_pattern.sub('', phrase).strip()
            
            # Écrire la phrase et les emojis dans le fichier CSV avec des guillemets autour
            csvwriter.writerow([clean_phrase, emojis[0]])

# Message de confirmation
print("Les phrases contenant des emojis ont été ajoutées au fichier 'phrases_emojis.csv'.")

Les phrases contenant des emojis ont été ajoutées au fichier 'phrases_emojis.csv'.
